In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 20.2 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 6.6 MB 65.7 MB/s 
     |████████████████████████████████| 596 kB 38.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [3]:
#loading the pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Obtaining blogs from the internet

In [9]:
URL = "https://towardsdatascience.com/deploy-any-ml-model-to-any-cloud-platform-f27a8311f6d4"


In [10]:
r = requests.get(URL)

In [11]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [12]:
ARTICLE

'Towards Data Science Jul 29 Save Deploy any ML Model to Any Cloud Platform Model serving isn’t just a hard problem, it’s a hard problem that constantly demands new solutions. Model serving, as part of MLOps, is the DevOps challenge of keeping a complicated, fragile artifact (the model) working in multiple dynamic environments. As frameworks are built and updated for training models, and production environments evolve for new capabilities and constraints, data scientists have to reimplement model serving scripts and rebuild model deployment processes. Data scientists working in large, well-resourced organizations can hand off their models to specialized MLOps teams for serving and deployment. But for those of us working at start-ups and newer companies, like I did for the first decade of my career, we had to handle the ML deployment challenge ourselves. The problem: serving and deploying a model requires an entirely different set of skills and technologies than training it did. A quick

In [13]:
#chunks
max_chunk = 500

In [14]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [15]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [16]:
len(chunks)

3

In [17]:
#Text summarization
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [18]:
res[0]

{'summary_text': ' Model serving, as part of MLOps, is the DevOps challenge of keeping a complicated, fragile artifact working in multiple dynamic environments . Data scientists working in large, well-resourced organizations can hand off their models to specialized MLOps teams for serving and deployment . But for those of us working at start-ups and newer companies, like I did for the first decade of my career, we had to handle the ML deployment challenge ourselves .'}

In [19]:
' '.join([summ['summary_text'] for summ in res])

' Model serving, as part of MLOps, is the DevOps challenge of keeping a complicated, fragile artifact working in multiple dynamic environments . Data scientists working in large, well-resourced organizations can hand off their models to specialized MLOps teams for serving and deployment . But for those of us working at start-ups and newer companies, like I did for the first decade of my career, we had to handle the ML deployment challenge ourselves .  Truss is an open-sourced version control tool to serve models locally and in production . Hugging Face is one of the many popular frameworks — also including LightGBM, PyTorch, scikit-learn, TensorFlow, and XGBoost — that Truss supports .  Truss offers a unified approach to model serving across model frameworks and deployment targets . Get started by starring the repo and working through the end-to-end deployment tutorial for your favorite framework and platform .'

In [20]:
text = ' '.join([summ['summary_text'] for summ in res])

In [21]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)